# Ejemplo generación de texto usando Redes Neuronales Recurrentes

Usando el libro de la divina comedia entrenaremos un modelo de lenguaje para que pueda generar oraciones en español.

In [1]:
import numpy as np
import torch



## Preprocesamiento del conjunto de datos

In [2]:
with open('pg57303.txt', 'r', encoding='utf8') as fp:
  text=fp.read()
start_indx = text.find('_LA DIVINA COMEDIA_')
end_indx = text.find('End of the Project Gutenberg')
text = text[start_indx:end_indx]
char_set = set(text)
print('Número de caracteres:', len(text))
print('Número de caracteres únicos', len(char_set))

Número de caracteres: 710326
Número de caracteres únicos 107


La mayoria de las librerías especializadas en redes neuronales e implementación de redes neuronales recurrentes no trabajan con entradas de formato string, por lo que debemos transformar el texto a formato numérico, para ello crearemos un dictionario en python que asigne a cada carácter a un entero, ```char2int```.



In [3]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)
text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32      )

print('Forma del texto codificado:', text_encoded.shape)
print(text[:20], '== codificación ==>', text_encoded[:20])
print(text_encoded[10:18], '== Invertido ==>', ''.join(char_array[text_encoded[10:18]]))


Forma del texto codificado: (710326,)
_LA DIVINA COMEDIA_
 == codificación ==> [56 39 28  1 31 36 49 36 41 28  1 30 42 40 32 31 36 28 56  0]
[ 1 30 42 40 32 31 36 28] == Invertido ==>  COMEDIA


Limitemos el tamaño de la secuencia a 40 caracteres. Además crearemos trozos de 41 caracteres dividiendo el texto original (40 caracteres que corresponden a las características y 1 caracter de etiqueta)

In [4]:
from torch.utils.data import Dataset
seq_length = 40
chunk_size = seq_length + 1
text_chunks = [text_encoded[i:i+chunk_size]
               for i in range(len(text_encoded)-chunk_size)]

class TextDataset(Dataset):
  def __init__(self, text_chunks):
    self.text_chunks = text_chunks

  def __len__(self):
    return len(self.text_chunks)

  def __getitem__(self, idx):
    text_chunk = self.text_chunks[idx]
    return text_chunk[:-1].long(), text_chunk[1:].long()

seq_dataset = TextDataset(torch.tensor(text_chunks))

<ipython-input-4-377d45116273>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  seq_dataset = TextDataset(torch.tensor(text_chunks))


In [5]:
for i, (seq, target) in enumerate(seq_dataset):
  print(' Input (x): ',
    repr(''.join(char_array[seq])))
  print('Target (y): ',
    repr(''.join(char_array[target])))
  print()
  if i == 1:
    break

 Input (x):  '_LA DIVINA COMEDIA_\n\n\n\n\n                '
Target (y):  'LA DIVINA COMEDIA_\n\n\n\n\n                 '

 Input (x):  'LA DIVINA COMEDIA_\n\n\n\n\n                 '
Target (y):  'A DIVINA COMEDIA_\n\n\n\n\n                  '



Finalmente el último paso para preparar este conjunto de datos es transformar el conjunto de datos en lotes

In [6]:
from torch.utils.data import DataLoader
batch_size = 64
torch.manual_seed(1)
seq_dl = DataLoader(seq_dataset, batch_size=batch_size,
shuffle=True, drop_last=True)

## Construyendo el modelo

In [7]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.rnn_hidden_size = rnn_hidden_size
    self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, batch_first=True)
    self.fc = nn.Linear(rnn_hidden_size, vocab_size)

  def forward(self, x, hidden, cell):
    out = self.embedding(x).unsqueeze(1)
    out, (hidden, cell) = self.rnn(out, (hidden, cell))
    out = self.fc(out).reshape(out.size(0), -1)
    return out, hidden, cell

  def init_hidden(self, batch_size):
    hidden = torch.zeros(1, batch_size, self.rnn_hidden_size)
    cell = torch.zeros(1, batch_size, self.rnn_hidden_size)
    return hidden, cell

Especificando los parametros del modelo:

In [8]:
vocab_size = len(char_array)
embed_dim = 256
rnn_hidden_size = 512
torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size)
model

RNN(
  (embedding): Embedding(107, 256)
  (rnn): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=107, bias=True)
)

Función de error y optimizador

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Entrenamiento

In [10]:
num_epochs = 10000
torch.manual_seed(1)
for epoch in range(num_epochs):
  hidden, cell = model.init_hidden(batch_size)
  seq_batch, target_batch = next(iter(seq_dl))
  optimizer.zero_grad()
  loss = 0
  for c in range(seq_length):
    pred, hidden, cell = model(seq_batch[:, c], hidden, cell)
    loss += loss_fn(pred, target_batch[:, c])
  loss.backward()
  optimizer.step()
  loss = loss.item()/seq_length
  if epoch % 500 == 0:
    print(f'Epoca {epoch+1} error: {loss:.4f}')

Epoca 1 error: 4.6840
Epoca 501 error: 1.7526
Epoca 1001 error: 1.5795
Epoca 1501 error: 1.4643
Epoca 2001 error: 1.4153
Epoca 2501 error: 1.4199
Epoca 3001 error: 1.3493
Epoca 3501 error: 1.2824
Epoca 4001 error: 1.2088
Epoca 4501 error: 1.2145
Epoca 5001 error: 1.1935
Epoca 5501 error: 1.1973
Epoca 6001 error: 1.1875
Epoca 6501 error: 1.1574
Epoca 7001 error: 1.1902
Epoca 7501 error: 1.1000
Epoca 8001 error: 1.0689
Epoca 8501 error: 1.0893
Epoca 9001 error: 1.1177
Epoca 9501 error: 1.1151


## Generación de texto

In [11]:
from torch.distributions.categorical import Categorical
torch.manual_seed(1)
logits = torch.tensor([[1.0, 1.0, 1.0]])
print('Probabilidades:',
  nn.functional.softmax(logits, dim=1).numpy()[0]
     )
m = Categorical(logits=logits)
samples = m.sample((10,))
print(samples.numpy())

Probabilidades: [0.33333334 0.33333334 0.33333334]
[[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [2]
 [1]
 [1]]


In [32]:
def sample(model, starting_str,len_generated_text=500,scale_factor=1.0):
  encoded_input = torch.tensor([char2int[s] for s in starting_str])
  encoded_input = torch.reshape(encoded_input, (1, -1))
  generated_str = starting_str

  model.eval()
  hidden, cell = model.init_hidden(1)
  for c in range(len(starting_str)-1):
    _, hidden, cell = model(encoded_input[:, c].view(1), hidden, cell)

  last_char = encoded_input[:, -1]
  for i in range(len_generated_text):
    logits, hidden, cell = model(last_char.view(1), hidden, cell)
    logits = torch.squeeze(logits, 0)
    scaled_logits = logits * scale_factor
    m = Categorical(logits=scaled_logits)
    last_char = m.sample()
    generated_str += str(char_array[last_char])

  return generated_str

In [33]:
torch.manual_seed(1)
print(sample(model, starting_str='eterno'))

eterno,
vuelves ante la misma noche pendiente seas, tus ojos del agua baja? ¿Por qué no fueron en
la inteligencia.
El rioble me hizo valle tan resistencia; y ahora te aún murió los años tan llenas de volor, gravedultes tan abandonar en no se nueva, la condive del objeto. Del que en su asciende a mi Guía, con una vió en este sitio donde yo
le suplique. Pero las fijés inyestrección suya donde se detuvo
al oír. Mas leemos descendido para nieve los hombres se regocijados los dos y tu mujeres bienaventurad
